# Example 1: Computation of Energies and Gradients

In this example it is shown, how to run the basic energy and gradient computations with novel approach for diabatization of orbitals.

First, we specify the geometry of our system.

In [1]:
symbols = ['C', 'N', 'H', 'H', 'H']
coords = [[-0.000000000, 0.000000000, -0.000000000],
            [-0.000000000, 0.000000000, 1.412052000],
            [-0.000000000, -0.944215264, -0.525349226],
            [-0.000000000, 0.944215264, -0.525349226],
            [-0.176643782, 0.000000000, 2.413848671]]

Now, let's define our system's properties.

In [2]:
n_orbs_active = 2
n_elec_active = 2
charge = 0
multiplicity = 1
basis = 'sto-3g'

For the diabatization process, we use previously obtained wavefunction. Let's load it now.

In [3]:
import psi4

load_wfn = psi4.core.Wavefunction.from_file(f'../tests/bruno_wfn_phi0_theta_180_sto3g')

   => Loading Basis Set <=

    Name: STO-3G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry C          line    61 file /home/martinb/anaconda3/envs/saoovqe-env/share/psi4/basis/sto-3g.gbs 
    atoms 2   entry N          line    71 file /home/martinb/anaconda3/envs/saoovqe-env/share/psi4/basis/sto-3g.gbs 
    atoms 3-5 entry H          line    19 file /home/martinb/anaconda3/envs/saoovqe-env/share/psi4/basis/sto-3g.gbs 



Now, we create our problem using our previously computed wavefunction, 

In [4]:
import saoovqe

problem = saoovqe.ProblemSet.with_dia_orbs_from_prev_wfn(symbols, coords, charge, multiplicity, n_elec_active,
                                                                 n_orbs_active, load_wfn, basis)


2024-06-21 02:12:01,601 - SAOOVQE.logger - INFO - SecondQuantizedProblem was created.


create initial circuits and ansatz,

In [5]:
n_states = 2
initial_circuits = saoovqe.OrthogonalCircuitSet.from_problem_set(n_states, problem)

repetitions = 1
ansatz = saoovqe.Ansatz.from_problem_set(saoovqe.AnsatzType.GUCCSD,
                                         problem,
                                         repetitions,
                                         qubit_mapper=problem.fermionic_mapper)

2024-06-21 02:12:01,655 - SAOOVQE.logger - INFO - Circuits representing an orthogonal basis were created.
2024-06-21 02:12:01,839 - SAOOVQE.logger - INFO - Ansatz was created.


and finally, we create our solver.

In [6]:
from qiskit.primitives import Estimator

estimator = Estimator()
saoovqe_solver = saoovqe.SAOOVQE(estimator,
                                 initial_circuits,
                                 ansatz,
                                 problem,
                                 orbital_optimization_settings={})

2024-06-21 02:12:01,842 - SAOOVQE.logger - INFO - SAOOVQE was created.


As we now have all that is required, we calculate our energies.

In [7]:
from qiskit_algorithms.optimizers import SciPyOptimizer

energies = saoovqe_solver.get_energy(SciPyOptimizer('SLSQP', options={'maxiter': 500, 'ftol': 1e-8}))
energies

2024-06-21 02:12:01,847 - SAOOVQE.logger - INFO - Computing energies...
--- Logging error ---
Traceback (most recent call last):
  File "/home/martinb/anaconda3/envs/saoovqe-env/lib/python3.12/logging/__init__.py", line 1160, in emit
    msg = self.format(record)
          ^^^^^^^^^^^^^^^^^^^
  File "/home/martinb/anaconda3/envs/saoovqe-env/lib/python3.12/logging/__init__.py", line 999, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "/home/martinb/anaconda3/envs/saoovqe-env/lib/python3.12/logging/__init__.py", line 703, in format
    record.message = record.getMessage()
                     ^^^^^^^^^^^^^^^^^^^
  File "/home/martinb/anaconda3/envs/saoovqe-env/lib/python3.12/logging/__init__.py", line 392, in getMessage
    msg = msg % self.args
          ~~~~^~~~~~~~~~~
TypeError: not all arguments converted during string formatting
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/

array([-92.69532965, -92.645587  ])

In [8]:
import numpy as np

np.allclose(np.array([-92.69532984620506, -92.64558680739152]), energies)

True

And we can see, that the result is correct!